# Burn a `mbtiles` file

#### Setup

_Assumes the following:_

- You have the AWS cli installed and configured with a firecares connection as the "default" profile that has:
    - The ability to invalidate CloudFront distributions
    - The ability to upload data to s3://firecares-data-backup
- You have GDAL/ogr2ogr installed
- You have tippecanoe installed
- You have the tessera-ansible project checked-out with the necessary ansible vault keys
- You have bash >= 4.0 installed
- You have plenty of space in your /tmp folder

**These cells will do the following:**

- Project the input file to EPSG:4326
- Upload the results to S3 under the county/state/location as provided
- Pull the remote geojson files from S3 that are used to create the .mbtiles output
- Generate mbtiles using tippecanoe from the collected geojson files
- Upload the output mbtiles to S3
- Re-download mbtiles and restart tileservers
- Invalidate the CloudFront tiles CDN => (a|b|c|d).firecares.org

In [20]:
# Location of the file to process
fname = '/tmp/Fire_Hydrants_9_8_2017.shp'

# One of `fire_hydrants`, `fire_stations`, `fire_districts`, `building_footprints`
feature_type = 'fire_hydrants'

# Meta about incoming file
info = dict(country='us', state='tx', location='hutto', feature_type=feature_type)

# Location of your tessera-ansible project
playbook_dir = '~/projects/firecares/tessera-ansible'

---

In [21]:
import os
file_root = os.path.splitext(fname)[0]
mbtiles_mapping = {
    'fire_districts': 'districts.mbtiles',
    'fire_hydrants': 'hydrants.mbtiles',
    'fire_stations': 'stations.mbtiles'
}
mbtiles_name = mbtiles_mapping[feature_type]
layer_name = mbtiles_name.split('.')[0]
dest = '{country}/{state}/{location}/{country}-{state}-{location}-{feature_type}.geojson'.format(**info)
outf = file_root + '.geojson'
s3dest = 's3://firecares-data-backup/' + dest

print s3dest

s3://firecares-data-backup/us/tx/hutto/us-tx-hutto-fire_hydrants.geojson


#### Copy projected geojson to S3

In [ ]:
!rm $outf
!ogr2ogr -f "Geojson" $outf $fname -t_srs EPSG:4326
!aws s3 cp $outf $s3dest

#### Copy feature geojson from S3 to local and flatten

In [22]:
%%bash -s "$feature_type"
rm -Rf /tmp/tiles
mkdir -p /tmp/tiles
cd /tmp/tiles
aws s3 cp s3://firecares-data-backup/ . --exclude="*" --include "*$1*.geojson" --exclude="*network_analysis*" --recursive
# You need bash >= 4.0 for globstar support => `brew install bash && echo '/usr/local/bin/bash' >> /etc/shells && chsh -s /usr/local/bin/bash`
shopt -s globstar
cd /tmp/tiles/us
cp */**/*.geojson ../

download: s3://firecares-data-backup/us/dc/washington/us-dc-washington-fire_hydrants.geojson to us/dc/washington/us-dc-washington-fire_hydrants.geojson
download: s3://firecares-data-backup/us/fl/tamarac/us-fl-tamarac-fire_hydrants.geojson to us/fl/tamarac/us-fl-tamarac-fire_hydrants.geojson
download: s3://firecares-data-backup/us/az/mesa/us-az-mesa-fire_hydrants.geojson to us/az/mesa/us-az-mesa-fire_hydrants.geojson
download: s3://firecares-data-backup/us/nc/new_bern/us-nc-new_burn-fire_hydrants.geojson to us/nc/new_bern/us-nc-new_burn-fire_hydrants.geojson
download: s3://firecares-data-backup/us/az/phoenix/us-az-phoenix-fire_hydrants.geojson to us/az/phoenix/us-az-phoenix-fire_hydrants.geojson
download: s3://firecares-data-backup/us/tx/hutto/us-tx-hutto-fire_hydrants.geojson to us/tx/hutto/us-tx-hutto-fire_hydrants.geojson
download: s3://firecares-data-backup/us/sc/kershaw_county/us-sc-kershaw_county-fire_hydrants.geojson to us/sc/kershaw_county/us-sc-kershaw_county-fire_hydrants.geoj

In [23]:
%%bash -s "$mbtiles_name" "$layer_name"
cd /tmp/tiles
echo "Writing layer: $2"
echo tippecanoe -r 0 -z18 -Z14 --no-line-simplification --no-feature-limit --no-tile-size-limit --no-polygon-splitting -f -l $2 -o $1 *.geojson
tippecanoe -r 0 -z18 -Z14 --no-line-simplification --no-feature-limit --no-tile-size-limit --no-polygon-splitting --include name -f -l $2 -o $1 *.geojson

Writing layer: hydrants
tippecanoe -r 0 -z18 -Z14 --no-line-simplification --no-feature-limit --no-tile-size-limit --no-polygon-splitting -f -l hydrants -o hydrants.mbtiles us-az-mesa-fire_hydrants.geojson us-az-phoenix-fire_hydrants.geojson us-ca-los_angeles_city-fire_hydrants.geojson us-ca-los_angeles_county-fire_hydrants.geojson us-dc-washington-fire_hydrants.geojson us-fl-tamarac-fire_hydrants.geojson us-il-chicago-fire_hydrants.geojson us-ks-kansas_city-fire_hydrants.geojson us-ma-boston-fire_hydrants.geojson us-nc-new_burn-fire_hydrants.geojson us-sc-kershaw_county-fire_hydrants.geojson us-tx-austin-fire_hydrants.geojson us-tx-houston-fire_hydrants.geojson us-tx-hutto-fire_hydrants.geojson us-tx-rowlett-fire_hydrants.geojson us-wi-la_crosse-fire_hydrants.geojson


us-ca-los_angeles_city-fire_hydrants.geojson:0: null geometry (additional not reported)
In JSON object {"type":"Feature","properties":{"HYD_NBR":80306,"CORNER":"SE","ADDRESS_NB":null,"STREET":"KESWICK AVE","CROSS_STRE":"BEVERLY GLEN BL","MAKE_CODE":"JONES","SIZE_CODE":"2 1/2 X4","MAIN_SIZE":6,"GATE_VALVE":6,"OUTLET_SIZ":6,"LATERAL_SI":6,"SUB_SURFAC":"n","CARTO_SYMB":1,"STATUS":0,"CALLOUT":null,"USECODE":0,"DATE_":"2014/11/18","EDITCODE":954,"SOURCECODE":2,"CHECKED":"jjz","OWNER_CODE":0,"PERMIT_NBR":null,"PRIVATE":"no","QAQC":null},"geometry":null}
us-tx-rowlett-fire_hydrants.geojson:0: ignoring dimensions beyond two
In JSON object [-96.531838077532726,32.961862558779629,0.0]
In JSON object {"type":"Feature","properties":{"MANUF_NAME":"Other","MANUF_DATE":"0","COLOR":"other","LOCATION_T":"Public","COMMENT":"No","STR_NUM":"3301","STR_NAME":"ELM GROVE","STR_TYPE":"RD","STREET_ADD":"3301 ELM GROVE RD","PROTYPE":"RES","Page":"083","Numbering":"001","LatDD":32.9618625588,"LongDD":-96.5318380

#### Upload mbiles to s3

In [24]:
%%bash -s "$mbtiles_name"
echo Copying $1 to S3
cd /tmp/tiles
aws s3 cp $1 s3://firecares-tiles/ --acl=public-read

Copying hydrants.mbtiles to S3
upload: ./hydrants.mbtiles to s3://firecares-tiles/hydrants.mbtiles


#### Re-download mbtiles on tile servers and restart tileserver process

In [25]:
!cd $playbook_dir && ansible-playbook -i hosts firecares-tileservers-production.yml --tags "mbtiles.reload" -e "reload_only=true" --private-key="~/.ssh/firecares-tileserver.pem"  


PLAY [Provision a tileserver tileserver] ************************************** 

GATHERING FACTS *************************************************************** 
ok: [107.22.2.98]
ok: [54.89.219.19]

TASK: [firecares | Stop application server] *********************************** 
changed: [54.89.219.19]
changed: [107.22.2.98]

TASK: [firecares | get_url url="https://s3.amazonaws.com/firecares-tiles/{{item}}" dest={{tiles_path}}/{{item}} force={{force_mbfiles_download|default('yes')}}] *** 
ok: [54.89.219.19] => (item=firecares_addresses.mbtiles)
ok: [107.22.2.98] => (item=firecares_addresses.mbtiles)
ok: [54.89.219.19] => (item=stations.mbtiles)
ok: [107.22.2.98] => (item=stations.mbtiles)
ok: [54.89.219.19] => (item=districts.mbtiles)
ok: [107.22.2.98] => (item=districts.mbtiles)
changed: [54.89.219.19] => (item=hydrants.mbtiles)
changed: [107.22.2.98] => (item=hydrants.mbtiles)

TASK: [firecares | Ensure that the application file permissions are set properly] *** 
ok: [54.89.219.19

#### Invalidate CDN

In [26]:
!aws cloudfront create-invalidation --distribution-id E3LIEGWQGOWXQG --paths '/*'

{
    "Invalidation": {
        "Status": "InProgress", 
        "InvalidationBatch": {
            "Paths": {
                "Items": [
                    "/*"
                ], 
                "Quantity": 1
            }, 
            "CallerReference": "cli-1520962859-791267"
        }, 
        "Id": "I3VF3AM2F29ZGP", 
        "CreateTime": "2018-03-13T17:40:59.745Z"
    }, 
    "Location": "https://cloudfront.amazonaws.com/2017-03-25/distribution/E3LIEGWQGOWXQG/invalidation/I3VF3AM2F29ZGP"
}


#### Preview

_Tested on node 4.2.2, see https://github.com/mojodna/tessera_

`tessera mbtiles://./hydrants.mbtiles`

### Burning current FireCARES station list

In [ ]:
import psycopg2
conn = psycopg2.connect('service=firecares')

sql = """
SELECT json_build_object(
    'type', 'FeatureCollection',
    'crs',  json_build_object(
        'type',      'name', 
        'properties', json_build_object(
            'name', 'urn:ogc:def:crs:OGC:1.3:CRS84')),
    'features', json_agg(
        json_build_object(
            'type',       'Feature',
            'id',         id,
            'geometry',   ST_AsGeoJSON(str.geom)::json,
            'properties', json_build_object(
                'name', str.name
            )
        )
    )
)
from firestation_firestation fs
inner join firestation_usgsstructuredata str
    on fs.usgsstructuredata_ptr_id = str.id
where fs.archived = false
    and fs.department_id is not null
"""

with conn.cursor() as c:
    c.execute(sql)
    res = c.fetchall()[0][0]

In [ ]:
import json
print len(res)
json.dump(res, open('/tmp/stations.geojson', 'w'), indent=4)

In [ ]:
!tippecanoe -r 0 -z18 -Z11 --no-line-simplification --no-feature-limit --no-tile-size-limit --no-polygon-splitting -f -l stations -o /tmp/stations.mbtiles /tmp/stations.geojson

In [ ]:
!aws s3 cp /tmp/stations.mbtiles s3://firecares-tiles/ --acl=public-read
!cd $playbook_dir && ansible-playbook -i hosts firecares-tileservers-production.yml --tags "mbtiles.reload" -e "reload_only=true" --private-key="~/.ssh/firecares-tileserver.pem"  
!aws cloudfront create-invalidation --distribution-id E3LIEGWQGOWXQG --paths '/*'